# Load Image Set

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive

In [6]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Check the directory structure to ensure the correct path
segmented_base_dir = "/content/drive/MyDrive/FYP/Lung/Segmented Images"

# Path to the directory containing segmented images
segmented_images_dir = os.path.join(segmented_base_dir, "Pneumothorax")
print(f"Using images directory: {segmented_images_dir}")

Using images directory: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax


In [8]:
# Check the directory structure to ensure the correct path
bone_suppressed_base_dir = "/content/drive/MyDrive/FYP/Lung/Bone Suppressed"

# Path to the directory containing segmented images
bone_suppressed_images_dir = os.path.join(bone_suppressed_base_dir, "Pneumothorax")
print(f"Using images directory: {bone_suppressed_images_dir}")

Using images directory: /content/drive/MyDrive/FYP/Lung/Bone Suppressed/Pneumothorax


# Pre Process the Images

In [ ]:
def preprocess_image(image_path):
   # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Step 1: Preprocess the image using GaussianBlur
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 2: Thresholding to create a binary image
    _, binary = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)
    return binary

# Identify Dark Lung Areas

In [13]:
def detect_combined_homogeneous_area(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Step 1: Preprocess the image using GaussianBlur
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 2: Thresholding to create a binary image
    _, binary = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)  # Adjusted threshold value

    # Step 3: Morphological operations to clean up the binary image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Step 4: Find contours in the binary image
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an output image to draw detected areas
    output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Function to check if a contour area is homogeneous
    def is_homogeneous_area(contour, image):
        mask = np.zeros_like(image)
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
        area_intensity = cv2.mean(image, mask=mask)[0]
        area_std_dev = np.std(image[mask == 255])
        return area_std_dev < 200  # Adjust the threshold for homogeneity

    # Function to separate low-contrast and high-contrast areas
    def separate_contrast_areas(contour, image):
        mask = np.zeros_like(image)
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
        mean_intensity = cv2.mean(image, mask=mask)[0]
        return mean_intensity < 50  # Adjust the threshold for separating contrast areas

    # Step 5: Filter and identify the homogeneous areas
    detected_areas = []
    for contour in contours:
        if is_homogeneous_area(contour, image) and separate_contrast_areas(contour, image):
            detected_areas.append(contour)

    # Step 6: Create a mask for the detected homogeneous areas
    mask = np.zeros_like(image)
    for contour in detected_areas:
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Step 7: Apply dilation followed by erosion to combine nearby areas
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))  # Adjust kernel size as needed
    dilated = cv2.dilate(mask, kernel, iterations=2)
    eroded = cv2.erode(dilated, kernel, iterations=2)

    # Step 8: Find the contour of the combined region
    combined_contours, _ = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the combined contour on the output image
    if combined_contours:
        cv2.drawContours(output_image, combined_contours, -1, (0, 255, 255), 2)

    return combined_contours, output_image

# Identify Visible Plural Line

In [14]:
def detect_pleural_line(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Step 1: Preprocess the image using GaussianBlur
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Step 2: Edge detection using Canny
    edges = cv2.Canny(blurred, 30, 150)  # Adjusted parameters for Canny edge detection

    # Step 3: Line detection using Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)

    # Create an output image to draw lines
    output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Function to check if a line is homogeneous
    def is_homogeneous_line(x1, y1, x2, y2, image):
        line_intensity = []
        num_points = max(abs(x2 - x1), abs(y2 - y1))
        for i in range(num_points + 1):
            x = int(x1 + i * (x2 - x1) / num_points)
            y = int(y1 + i * (y2 - y1) / num_points)
            line_intensity.append(image[y, x])

        if len(line_intensity) < 2:
            return False

        # Calculate the standard deviation of the intensities along the line
        std_dev = np.std(line_intensity)
        # Check if the line intensity is relatively consistent (low standard deviation)
        return std_dev < 20

    # Function to check if a line separates dark and light areas
    def separate_dark_light_areas(x1, y1, x2, y2, image):
        midpoint_x = (x1 + x2) // 2
        midpoint_y = (y1 + y2) // 2
        left_intensity = np.mean(image[max(0, midpoint_y - 5):midpoint_y + 5, max(0, midpoint_x - 20):midpoint_x])
        right_intensity = np.mean(image[max(0, midpoint_y - 5):midpoint_y + 5, midpoint_x:min(image.shape[1], midpoint_x + 20)])
        return abs(left_intensity - right_intensity) > 50

    # Step 4: Filter and identify the pleural line
    detected_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if is_homogeneous_line(x1, y1, x2, y2, image) and separate_dark_light_areas(x1, y1, x2, y2, image):
                # Draw the detected pleural line on the output image
                cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                detected_lines.append((x1, y1, x2, y2))

    return detected_lines

# Predict Pneumothorax

In [15]:
def predict_pneumothorax(segmented_image_path):
    combined_contours, output_image = detect_combined_homogeneous_area(segmented_image_path)

    if combined_contours:
        pleural_lines = detect_pleural_line(segmented_image_path)
        if pleural_lines:
            for line in pleural_lines:
                x1, y1, x2, y2 = line
                # Check if the pleural line is near the homogeneous area
                for contour in combined_contours:
                    if cv2.pointPolygonTest(np.array(contour, dtype=np.float32), ((x1 + x2) / 2, (y1 + y2) / 2), False) >= 0:
                        print(f"Pneumothorax is suspected in image: {segmented_image_path}")
                        return True

    print(f"No significant pleural lines detected, No Pneumothorax suspected in image: {segmented_image_path}")
    return False

In [16]:
def iterate_through_images(directory):
    files = os.listdir(directory)
    image_files = [file for file in files if file.endswith('.jpg') or file.endswith('.png')]
    results = []
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        result = predict_pneumothorax(image_path)
        results.append(result)
    return results

# Directory containing the segmented lung images
image_directory = '/content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax'

# Get the detection results for all images
detection_results = iterate_through_images(image_directory)


Pneumothorax is suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_104.png
No significant pleural lines detected, No Pneumothorax suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_113.png
No significant pleural lines detected, No Pneumothorax suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_105.png
No significant pleural lines detected, No Pneumothorax suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_83.png
Pneumothorax is suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_96.png
Pneumothorax is suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_56.png
No significant pleural lines detected, No Pneumothorax suspected in image: /content/drive/MyDrive/FYP/Lung/Segmented Images/Pneumothorax/segmented_107.png
Pneumothorax is suspected in image:

In [17]:
# Calculate accuracy
TP = sum(detection_results)  # Number of True Positives
all_images = len(detection_results)  # Total number of images

accuracy = TP / all_images if all_images > 0 else 0
print('Accuracy:', accuracy)

Accuracy: 0.5977011494252874
